In [2]:
import pandas as pd
import numpy as np
import datetime
import os
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)

In [3]:
df_original_reports = pd.read_csv("data-processed/rki-reports.csv")
df_apple = pd.read_csv("data-processed/data_apple_prepared.csv")
df_rki_orig = pd.read_csv('dash/data/data_rki_apple_prepared_dash.csv')
df_jh_world = pd.read_csv('dash/data/data_jhu_world.csv')
# df = pd.read_csv("dash/data/data_rki_apple_prepared_dash.csv")
df_rki_orig['date']=df_rki_orig['date'].astype('datetime64[ns]')
df_rki_orig.index = df_rki_orig.date
df_jh_world['date']=df_jh_world['date'].astype('datetime64[ns]')
df_jh_world.index = df_jh_world.date

In [737]:
# df_original_reports #.loc[df_original_reports.date != '2020-05-28'].drop("Unnamed: 0", axis=1).to_csv("data-processed/rki-reports.csv", index=False)

In [1026]:
import pyarrow as pa
import redis

redis_conf = {'host': 'localhost', "port": 6379, 'db': 0}

redis_pool = None

def init():
    global redis_pool
    print("PID %d: initializing redis pool..." % os.getpid())
    redis_pool = redis.ConnectionPool(host=redis_conf['host'], port=redis_conf['port'], db=redis_conf['db'])

def cache_df(alias,df):

    cur = redis.Redis(connection_pool=redis_pool)
    context = pa.default_serialization_context()
    df_compressed =  context.serialize(df).to_buffer().to_pybytes()

    res = cur.set(alias,df_compressed)
    if res == True:
        print('df cached')

In [1027]:
init()

PID 4931: initializing redis pool...


In [1028]:
cache_df('df_test', df_jh_world)

df cached


In [6]:
def get_cached_df(alias):

    cur = redis.Redis(connection_pool=redis_pool)
    context = pa.default_serialization_context()
    all_keys = [key.decode("utf-8") for key in cur.keys()]

    if alias in all_keys:   
        result = cur.get(alias)

        dataframe = pd.DataFrame.from_dict(context.deserialize(result))

        return dataframe

    return None

In [1036]:
# pool = redis.ConnectionPool(redis_conf['host'], port=redis_conf['port'], db=redis_conf['db']) 
# cur = redis.Redis(connection_pool=pool)

In [1037]:
# cl = redis.Redis(redis_conf['host'], port=redis_conf['port'], db=redis_conf['db'])
# cl.set('hello', 'world')

In [1038]:
# cl.get('hello')

In [1990]:
# df = get_cached_df('df_test')